In [15]:
# import utility modules
import pandas as pd
import numpy as np
import configparser
import os

import numpy as np
import pandas as pd
import latex

from scipy.optimize import minimize
from helpers.helper_classes import AddFeatureNames, Gene_SPCA
from helpers.helper_functions import get_data_pev
from joblib import dump, load

import matplotlib.pyplot as plt
import scienceplots
plt.style.use(['science', 'ieee'])

In [21]:
# Read config.ini file
config = configparser.ConfigParser()
config.read('src/config.ini')

os.chdir(config['PATH']['ROOT_DIR'])

# Read data
data = load(config['PATH']['DATA_DIR'] + '/microarray-data-dict.lib')
os.environ["PATH"] += os.pathsep + '/Library/TeX/texbin'

# Read parameters
SEED = config.getint('PARAMS', 'SEED')
N_COMPONENTS = config.getint('PARAMS', 'N_COMPONENTS')

In [17]:
X = data['gravier']['none']['X_train']

In [18]:
# Golub because original, Christensen because of small dataset, Chin because of large dataset, Nakayama because of large number of classes
pev_dataset_list = ['golub', 'christensen', 'chin', 'nakayama']

In [22]:
nz_cols, nz_loads, pev_gravier = get_data_pev(X, n_components = 20, verbose = 1)
nz_cols2, nz_loads2, pev_christensen = get_data_pev(data['christensen']['none']['X_train'], n_components = 20, verbose = 1)

l1 =  0
nonzero_cnt =  2905
nonzero_loadings =  58100
PEV =  0.14026994038425428



KeyboardInterrupt: 

In [8]:
# Plot nz_cols and nz_cols2 vs pev_gravier and pev_christensen
plt.plot(nz_loads2, pev_christensen)
plt.plot(nz_loads, pev_gravier, label = 'Gravier')


NameError: name 'nz_loads2' is not defined